In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, LabelEncoder, OneHotEncoder
from datetime import datetime
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)
import folium
from folium.plugins import HeatMap, MousePosition
import random
# from branca.element import Template, MacroElement
from geopy.distance import geodesic
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, BayesianRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error, mean_squared_log_error, make_scorer

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', 290)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [9]:
df_ = pd.read_csv('hemnet.csv')

In [10]:
df = df_.copy()

In [11]:
df_ = pd.read_csv("properties.csv")

In [ ]:
deneme = pd.read_csv("hemnet_last.csv")
deneme.head(100)

In [ ]:
# 'sold_date' değişkenini tarih tipine çevir
deneme['sold_date'] = pd.to_datetime(deneme['sold_date'])

# 'sold_date' değişkeninden gün bilgilerini al ve yeni bir sütun olarak ekle
deneme['day'] = deneme['sold_date'].dt.day
deneme.head()

In [ ]:
deneme.drop(columns = ['sold_date'], axis = 1, inplace = True)
deneme.head()


In [ ]:
deneme.rename(columns={'day': 'sold_date'}, inplace=True)
deneme.head()

In [ ]:
translation_dict={
'Villa':'House',
'Lägenhet':'Apartment',
'Tomt':'Plot',
'Radhus':'Town house',
'Parhus':'Terraced house',
'Vinterbonatfritidshus':'Winterized holiday home',
'Övrig':'Other',
'Kedjehus':'Linked house',
'Par-/kedje-/radhus':'Row/Terrace/Town house',
'Fritidshus':'Holiday home',
'Gårdutanjordbruk':'Farm without agriculture',
'Gård/skog':'Farm/Forest',
'Gårdmedjordbruk':'Farm with agriculture',
'Fritidsboende':'Holiday accommodation',
'Gårdmedskogsbruk':'Farm with forestry'
}


deneme['property_type_en']=deneme['property_type'].map(translation_dict)
deneme=deneme.drop(["property_type"],axis=1)
deneme.head()


In [ ]:


translation_dict={
'Äganderätt':'Freehold',
'Bostadsrätt':'Cooperative apartment',
'Annat':'Other',
'Andelsboende':'Cooperative housing',
'Tomträtt':'Landlease',
'Andelibostadsförening':'Share in housing association',
'Informationsaknas':'Information missing',
'Hyresrätt':'Rental'
}

#Çeviriyiuygula
deneme['ownership_type_en']=deneme['ownership_type'].map(translation_dict)
deneme=deneme.drop(["ownership_type"],axis=1)
deneme.head()

In [7]:



#Sütunadınıdeğiştirme
deneme.rename(columns={'broker':'estate_agent',
'property_type_en':'property_type',
'ownership_type_en':'ownership_type'},inplace=True)
deneme.head()

In [ ]:
deneme['balcony'].head(100)
# 'balcony' değişkenini çevir
deneme['balcony'] = deneme['balcony'].map({'Ja': 'Yes', 'Nej': 'No'})

print(deneme)

In [ ]:
deneme.to_csv('hemnet_last2.csv', index = False)


In [ ]:
deneme2 = pd.read_csv("hemnet_last2.csv")
deneme2.head()

In [ ]:
# Unix zaman damgasını datetime formatına çevirin
df_['sold_at'] = pd.to_datetime(df_['sold_at'], unit='s')

# Yıl, ay ve gün bileşenlerini çıkartın
df_['sold_year'] = df_['sold_at'].dt.year
df_['sold_month'] = df_['sold_at'].dt.month
df_['sold_date'] = df_['sold_at'].dt.day
df_.drop(columns = ['sold_at'], axis = 1, inplace = True)
df_.head()

In [ ]:
df_.head()
df_['construction_year'] = pd.to_datetime(df_['construction_date'], errors='coerce').dt.year
df_.drop(columns=['construction_date'], axis=1, inplace=True)

In [ ]:
df_['build_year'] = pd.to_numeric(df_['build_year'], errors='coerce')
df_['build_year'] =df_['build_year'].astype(int)
df_['build_year'] = pd.to_datetime(df_['build_year'], format='%Y', errors='coerce')

df = df_.copy()

In [ ]:
df.head()

In [ ]:

# df.to_csv('hemnet2.csv', index = False)

def check_df(dataframe: pd.DataFrame, head: int = 5, q1: float = 0.05, q2: float = 0.50, q3: float = 0.95, q4: float = 0.99):
    """
    Display comprehensive information about the dataframe.

    Args:
        dataframe (pd.DataFrame): The input dataframe to analyze.
        head (int): Number of rows to display for head and tail. Default is 5.
        q1 (float): First quantile value. Default is 0.05.
        q2 (float): Second quantile value. Default is 0.50.
        q3 (float): Third quantile value. Default is 0.95.
        q4 (float): Fourth quantile value. Default is 0.99.

    Raises:
        ValueError: If the input is not a pandas DataFrame.
    """
    if not isinstance(dataframe, pd.DataFrame):
        raise ValueError("Input must be a pandas DataFrame")

    def print_section(title):
        print(f"\n{'-' * 20} {title} {'-' * 20}\n")

    print_section("Shape of the dataframe")
    print(f"Rows: {dataframe.shape[0]}, Columns: {dataframe.shape[1]}")

    print_section("Data Types")
    print(dataframe.dtypes)

    print_section("Head")
    print(dataframe.head(head))

    print_section("Tail")
    print(dataframe.tail(head))

    print_section("Missing Values")
    missing = dataframe.isnull().sum()
    missing_pct = 100 * dataframe.isnull().sum() / len(dataframe)
    missing_table = pd.concat([missing, missing_pct], axis=1, keys=['Total', 'Percent'])
    print(missing_table[missing_table['Total'] > 0].sort_values('Total', ascending=False))

    print_section("Descriptive Statistics")
    datetime_columns = dataframe.select_dtypes(include='datetime64[ns]').columns
    desc = dataframe.drop(datetime_columns, axis = 1).describe([0, q1, q2, q3, q4, 1]).T
    desc['range'] = desc['max'] - desc['min']
    desc['coef_var'] = desc['std'] / desc['mean']
    print(desc)

    print_section("Skewness")
    skew = dataframe.select_dtypes(include='number').skew().sort_values(ascending=False)
    print(skew)

    print_section("Kurtosis")
    kurt = dataframe.select_dtypes(include='number').kurt().sort_values(ascending=False)
    print(kurt)

    print_section("Correlation Matrix")
    corr = dataframe.select_dtypes(include='number').corr()
    print(corr)

    print_section("Memory Usage")
    memory_usage = dataframe.memory_usage(deep=True)
    total_memory = memory_usage.sum()
    print(f"Total Memory Usage: {total_memory / 1e6:.2f} MB")
    print(memory_usage)

    print_section("Unique Values")
    for col in dataframe.columns:
        n_unique = dataframe[col].nunique()
        print(f"{col}: {n_unique} unique values")

    print_section("Sample Data")
    print(dataframe.sample(min(5, len(dataframe))))

check_df(df)

In [ ]:

def grab_col_names(dataframe, cat_th=10, car_th=300):
    """
    Grab column names based on their types and cardinality.

    Parameters:
    cat_th (int): Threshold for numerical columns to be considered categorical. Default is 10.
    car_th (int): Threshold for categorical columns to be considered cardinal. Default is 20.

    Returns:
    tuple: Lists of categorical columns, numerical columns, and cardinal columns.
    """
    cat_cols = [col for col in dataframe.columns if str(dataframe[col].dtypes) in ['category', 'object', 'bool']]
    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and dataframe[col].dtypes in ['int', 'float']]
    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() > car_th and str(dataframe[col].dtypes) in ['category', 'object']]
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]
    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes in ['int', 'float', 'int64', 'float64']]
    num_cols = [col for col in num_cols if col not in cat_cols]
    date_cols = [col for col in dataframe.columns if dataframe[col].dtypes == 'datetime64[ns]']

    return cat_cols, num_cols, cat_but_car, date_cols

cat_cols, num_cols, cat_but_car, date_cols = grab_col_names(df, cat_th = 10, car_th = 300)

In [ ]:

def reporting_col_types(dataframe):
    """
    Report the types of columns in the dataframe.
    """
    cat_cols, num_cols, cat_but_car, date_cols = grab_col_names(dataframe, cat_th = 10, car_th = 300)
    print(f"Observations: {dataframe.shape[0]}\n")
    print(f"Variables: {dataframe.shape[1]}\n")
    print(f'cat_cols: {len(cat_cols)}\n')
    print(f'num_cols: {len(num_cols)}\n')
    print(f'cat_but_car: {len(cat_but_car)}\n')
    print(f'date_cols: {len(date_cols)}\n')
    print(f'The categoric columns are: {cat_cols}\n')
    print(f'The numerical columns are: {num_cols}\n')
    print(f'The categorical but cardinal columns are: {cat_but_car}\n')
    print(f'The date columns are: {date_cols}\n')

reporting_col_types(df)

In [ ]:

def value_counts_ratio(df, columns, plot=False, verbose=False):
    """
    Calculate the value counts and their proportions for specified columns in a DataFrame, and optionally plot pie charts and bar plots.
    Parameters:
    - df (pd.DataFrame): The DataFrame to analyze.
    - columns (list of str): List of column names to calculate value counts for.
    - plot (bool): If True, plot pie charts and bar plots for each column. Default is False.
    - verbose (bool): If True, print the value counts and percentages. Default is False.
    Returns:
    - value_counts_dfs (dict of pd.DataFrame): Dictionary of DataFrames containing value counts and percentages for each column.
    """
    if not isinstance(df, pd.DataFrame):
        raise ValueError("Input must be a pandas DataFrame")
    if not all(isinstance(col, str) for col in columns):
        raise ValueError("Columns must be a list of strings")

    value_counts_dfs = {}
    for col in columns:
        if col not in df.columns:
            raise ValueError(f"Column '{col}' not found in DataFrame")

        value_counts_df = df[col].value_counts(normalize=True).to_frame().reset_index()
        value_counts_df.columns = ['value', 'percentage']
        value_counts_dfs[col] = value_counts_df

        if verbose:
            print(f"\nColumn: {col}")
            print(value_counts_df)

        if plot:
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
            fig.suptitle(f'{col.capitalize()} Proportions', fontsize=16)

            def autopct_format(pct):
                return f'{pct:.1f}%' if pct >= 2 else ''

            wedges, texts, autotexts = ax1.pie(
                value_counts_df['percentage'],
                labels=None,
                autopct=autopct_format,
                colors=sns.color_palette("tab10", len(value_counts_df)),
                textprops={'fontsize': 10}
            )
            ax1.set_title('Pie Chart', fontsize=14)
            ax1.legend(wedges, value_counts_df['value'], title="Values", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))

            sns.barplot(x='value', y='percentage', data=value_counts_df, ax=ax2, palette="tab10")
            ax2.set_title('Bar Plot', fontsize=14)
            ax2.set_xlabel('')
            ax2.set_ylabel('Percentage')
            ax2.set_xticklabels([])

            plt.tight_layout()
            plt.show()

    return value_counts_dfs

In [ ]:

value_counts_ratio(df, ['property_type','housing_form', 'balcony', 'sold_year', 'sold_month', 'ownership_type'] ,plot = True)

In [ ]:

value_counts_dfs = value_counts_ratio(df, ['property_type','housing_form', 'balcony', 'sold_year', 'sold_month', 'ownership_type'])

In [ ]:

value_counts_dfs['ownership_type']

In [ ]:


def null_dataframe(dataframe, plot=False, verbose=False):

    """
    Analyze and visualize null values in a DataFrame.
    Parameters:
    - dataframe (pd.DataFrame): The DataFrame to analyze.
    - cat_cols (list of str): List of categorical columns.
    - num_cols (list of str): List of numerical columns.
    - cat_but_car (list of str): List of categorical but cardinal columns.
    - date_cols (list of str): List of date columns.
    - plot (bool): Whether to plot charts for variables with null values. Default is False.
    - verbose (bool): If True, print additional information. Default is False.
    Returns:
    - df_null (pd.DataFrame): DataFrame containing variables, their null value counts, and null ratios.
    """

    if not isinstance(dataframe, pd.DataFrame):
        raise ValueError("Input must be a pandas DataFrame")

    df_null = dataframe.isnull().sum().sort_values(ascending=False).to_frame(name='null_values')
    df_null.reset_index(inplace=True)
    df_null.rename(columns={'index': 'variables'}, inplace=True)

    variable_types = {col: 'Categorical' for col in cat_cols}
    variable_types.update({col: 'Numerical' for col in num_cols})
    variable_types.update({col: 'Categorical Cardinal' for col in cat_but_car})
    variable_types.update({col: 'Date' for col in date_cols})

    df_null['variable_type'] = df_null['variables'].map(variable_types)
    df_null['null_ratio'] = df_null['null_values'] / len(dataframe)

    null_variables = df_null[df_null['null_values'] != 0]['variables'].tolist()
    notnull_variables = df_null[df_null['null_values'] == 0]['variables'].tolist()

    if verbose:
        print(f'The null variables are: {null_variables}\n')
        print(f'The not null variables are: {notnull_variables}\n')
        print(df_null)

    if plot and null_variables:
        for var in null_variables:
            null_ratio = df_null[df_null['variables'] == var]['null_ratio'].values[0]

            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
            fig.suptitle(f'Null Ratio for {var.capitalize()} Variable', fontsize=16)

            def autopct_format(pct):
                return f'{pct:.1f}%' if pct >= 2 else ''

            wedges, texts, autotexts = ax1.pie(
                [null_ratio, 1 - null_ratio],
                labels=None,
                autopct=autopct_format,
                colors=['#636EFA', '#EF553B'],
                textprops={'fontsize': 10}
            )
            ax1.set_title('Pie Chart', fontsize=14)
            ax1.legend(wedges, ['Null', 'Not Null'], title="Values", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))

            sns.barplot(x=['Null', 'Not Null'], y=[null_ratio, 1 - null_ratio], ax=ax2, palette=['#636EFA', '#EF553B'])
            ax2.set_title('Bar Plot', fontsize=14)
            ax2.set_xlabel('')
            ax2.set_ylabel('Ratio')
            ax2.set_xticklabels([])

            plt.tight_layout()
            plt.show()

    return df_null

In [ ]:

null_dataframe(df, plot = True)

In [ ]:

null_df  = null_dataframe(df)

In [ ]:

null_df[:13]

In [ ]:

num_cols

In [ ]:

def zero_values_ratio(dataframe, plot=False, verbose=False):
    """
    Analyze and visualize zero values in numerical columns of a DataFrame.
    Parameters:
    - dataframe (pd.DataFrame): The DataFrame to analyze.
    - num_cols (list of str): List of numerical columns.
    - plot (bool): Whether to plot charts for variables with zero values. Default is False.
    - verbose (bool): If True, print additional information. Default is False.
    Returns:
    - zero_values_df (pd.DataFrame): DataFrame containing variables, their zero value counts, and zero ratios.
    """
    zero_values_data = []

    for col in num_cols:
        zero_count = dataframe[dataframe[col] == 0].shape[0]
        if zero_count != 0:
            zero_ratio = zero_count / dataframe.shape[0]
            zero_values_data.append({
                'Variable': col,
                'Zero_Count': zero_count,
                'Zero_Ratio': zero_ratio
            })

            if verbose:
                print(f'{col.capitalize()} column has {zero_count} zero values, ratio to all dataframe: {zero_ratio:.5f}')
                print('\n')

            if plot:
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
                fig.suptitle(f'Zero Ratio for {col.capitalize()} Variable', fontsize=16)

                def autopct_format(pct):
                    return f'{pct:.1f}%' if pct >= 2 else ''

                wedges, texts, autotexts = ax1.pie(
                    [zero_ratio, 1 - zero_ratio],
                    labels=None,
                    autopct=autopct_format,
                    colors=['#636EFA', '#EF553B'],
                    textprops={'fontsize': 10}
                )
                ax1.set_title('Pie Chart', fontsize=14)
                ax1.legend(wedges, ['Zero', 'Not Zero'], title="Values", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))

                sns.barplot(x=['Zero', 'Not Zero'], y=[zero_ratio, 1 - zero_ratio], ax=ax2, palette=['#636EFA', '#EF553B'])
                ax2.set_title('Bar Plot', fontsize=14)
                ax2.set_xlabel('')
                ax2.set_ylabel('Ratio')
                ax2.set_xticklabels([])

                plt.tight_layout()
                plt.show()

            if verbose:
                print('\n')

    zero_values_df = pd.DataFrame(zero_values_data)

    if verbose:
        print(zero_values_df)

    return zero_values_df

In [ ]:

zero_values_ratio(df, plot = True)

In [ ]:

zero_values_df = zero_values_ratio(df)

In [ ]:

zero_values_df.sort_values('Zero_Count', ascending = False)

In [ ]:

df[df['latitude'] == 0]

In [ ]:

df[df['longitude'] == 0]

In [ ]:

df = df[df['latitude'] != 0]

In [ ]:

df.iloc[208]

In [ ]:

df[df['living_area'] == 0]['housing_form'].value_counts()

In [ ]:

df[df['living_area'] == 0]['housing_form'].value_counts()

In [ ]:

def replace_zeros_with_mean(dataframe, column, target):
    mean_values = dataframe.groupby(column)[target].transform(lambda x: x.replace(0, x.mean()))
    dataframe[target] = dataframe[target].where(dataframe[target] != 0, mean_values)

    return dataframe

    return dataframe

In [ ]:

def replace_zeros_with_median(dataframe, column, target):
    median_values = dataframe.groupby(column)[target].transform(lambda x: x.replace(0, x.median()))
    dataframe[target] = dataframe[target].where(dataframe[target] != 0, median_values)

    return dataframe

In [ ]:

def replace_nulls(dataframe, column, target, method='median', verbose = False):
    if column not in dataframe.columns or target not in dataframe.columns:
        raise ValueError("Specified column or target not found in dataframe.")

    if method == 'median':
        replacement_values = dataframe.groupby(column)[target].transform('median')
    elif method == 'mean':
        replacement_values = dataframe.groupby(column)[target].transform('mean')
    elif method == 'mode':
        replacement_values = dataframe.groupby(column)[target].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
    else:
        raise ValueError("Invalid replacement method. Please choose 'median', 'mean', or 'mode'.")

    if verbose:
        print("Before filling nulls:")
        print(dataframe[target].isnull().sum())
        print("Replacement values computed:")
        print(replacement_values)

    dataframe[target] = dataframe[target].fillna(replacement_values)

    if verbose:
        print("After filling nulls:")
        print(dataframe[target].isnull().sum())

    return dataframe

In [ ]:

null_df = null_dataframe(df)

In [ ]:

null_df

In [ ]:

zero_values_df = zero_values_ratio(df)

In [ ]:

zero_values_df

In [ ]:

df[df['county'].isnull()]

In [ ]:

df_coordinates_county = df.groupby('county')[['latitude', 'longitude']].mean().reset_index()
df_coordinates_county[:20]

In [ ]:

target_coords_1 = (59.31968, 18.06227)
target_coords_2 = (59.23744, 18.12341)
target_coords_3 = (59.44499, 13.43092)

In [ ]:

df_coordinates_county['distance1'] = df_coordinates_county.apply(lambda row: geodesic((row['latitude'], row['longitude']), target_coords_1).kilometers, axis = 1)
df_coordinates_county['distance2'] = df_coordinates_county.apply(lambda row: geodesic((row['latitude'], row['longitude']), target_coords_2).kilometers, axis = 1)
df_coordinates_county['distance3'] = df_coordinates_county.apply(lambda row: geodesic((row['latitude'], row['longitude']), target_coords_3).kilometers, axis = 1)

In [ ]:

closest_county_1 = df_coordinates_county.loc[df_coordinates_county['distance1'].idxmin()]
closest_county_2 = df_coordinates_county.loc[df_coordinates_county['distance2'].idxmin()]
closest_county_3 = df_coordinates_county.loc[df_coordinates_county['distance3'].idxmin()]

In [ ]:

closest_county_1, closest_county_2, closest_county_3

In [ ]:

df.loc[156981, 'county'] = 'Stockholms'
df.loc[938636, 'county'] = 'Huddinge'
df.loc[1084895, 'county'] = 'Karlstads'

In [ ]:

df['street'].isnull().sum()

In [ ]:

df['street'].value_counts()

In [ ]:

df = replace_nulls(df, 'county', 'street', method = 'mode')
# df = replace_nulls(df, 'county', 'price_change', method = 'median')
df = replace_nulls(df, 'county', 'area', 'mode')
df = replace_zeros_with_mean(df, 'housing_form', 'living_area')
df = replace_zeros_with_mean(df, 'county', 'fee')
df = replace_zeros_with_mean(df, 'county', 'wanted_price')
df = replace_zeros_with_mean(df, 'housing_form', 'rooms')

In [ ]:

df[df['fee'] == 0]

In [ ]:

df.loc[df['fee'] == 0, 'fee'] = df['fee'].median()

In [ ]:

#df.to_csv('hemnet3.csv', index = False)

def create_county_markers(lats, lons, counties, location=[62.0, 15.0], zoom_start=4):
    """
    Create a map with circle markers for given counties and their coordinates.

    Parameters:
    - lats (list or pd.Series): List or Series of latitudes.
    - lons (list or pd.Series): List or Series of longitudes.
    - counties (list or pd.Series): List or Series of county names.
    - location (list of float): Initial map center coordinates [latitude, longitude]. Default is [62.0, 15.0].
    - zoom_start (int): Initial map zoom level. Default is 4.

    Returns:
    - sweden_map (folium.Map): Folium map object with the county markers.
    """

    if not isinstance(lats, (list, pd.Series)):
        raise ValueError("Lats must be a list or a pandas Series")
    if not isinstance(lons, (list, pd.Series)):
        raise ValueError("Lons must be a list or a pandas Series")
    if not isinstance(counties, (list, pd.Series)):
        raise ValueError("Counties must be a list or a pandas Series")
    if len(lats) != len(lons) or len(lats) != len(counties):
        raise ValueError("Lats, lons, and counties must have the same length")

    if isinstance(lats, pd.Series):
        lats = lats.tolist()
    if isinstance(lons, pd.Series):
        lons = lons.tolist()
    if isinstance(counties, pd.Series):
        counties = counties.tolist()

    sweden_map = folium.Map(location=location, zoom_start=zoom_start)

    colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue',
              'darkgreen', 'cadetblue', 'darkpurple', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']

    for county, lat, lon in zip(counties, lats, lons):
        color = random.choice(colors)
        folium.CircleMarker(
            location=[lat, lon],
            radius=7,
            popup=f"County: {county},  Lat: {lat}, Lon: {lon}",
            color=color,
            fill=True,
            fillColor=color
        ).add_to(sweden_map)
    return sweden_map

In [ ]:

sweden_map = create_county_markers(df_coordinates_county['latitude'], df_coordinates_county['longitude'], df_coordinates_county['county'])
sweden_map.save('sweden_map.html')

In [ ]:

df_coordinates_county_price = df.groupby('county')[['latitude', 'longitude', 'price']].mean().sort_values('price', ascending=False).reset_index()
df_coordinates_county_price[:20]

In [ ]:

def create_price_heatmap(dataframe):
    """
    Create an interactive heatmap based on price values using given coordinates grouped by county.
    Includes a mouse position display showing coordinates, markers for the top 3 most expensive counties,
    and legends for both the markers and the heatmap colors.

    Parameters:
    - dataframe (pd.DataFrame): DataFrame containing 'county', 'latitude', 'longitude', and 'price' columns.

    Returns:
    - folium.Map: Folium map object with the heatmap, mouse position display, markers, and legends.
    """

    if not all(col in dataframe.columns for col in ['county', 'latitude', 'longitude', 'price']):
        raise ValueError("DataFrame must contain 'county', 'latitude', 'longitude', and 'price' columns")

    map_center = [dataframe['latitude'].mean(), dataframe['longitude'].mean()]
    folium_map = folium.Map(location=map_center, zoom_start=6)

    heat_data = dataframe[['latitude', 'longitude', 'price']].values.tolist()
    HeatMap(heat_data).add_to(folium_map)

    formatter = "function(num) {return L.Util.formatNum(num, 5) + ' º ';};"
    mouse_position = MousePosition(
        position='topright',
        separator=' | ',
        empty_string='NaN',
        lng_first=True,
        num_digits=20,
        prefix="Coordinates:",
        lat_formatter=formatter,
        lng_formatter=formatter
    )
    folium_map.add_child(mouse_position)

    top_3_counties = dataframe.groupby('county')['price'].mean().nlargest(3).reset_index()

    colors = ['red', 'darkred', 'orange']
    for idx, row in top_3_counties.iterrows():
        county_data = dataframe[dataframe['county'] == row['county']].iloc[0]
        folium.Marker(
            location=[county_data['latitude'], county_data['longitude']],
            popup=f"County: {row['county']}<br>Avg Price: ${row['price']:,.2f}",
            icon=folium.Icon(color=colors[idx], icon='star', prefix='fa')
        ).add_to(folium_map)

    legend_html = '''
     <div style="
     position: fixed;
     bottom: 50px; left: 50px; width: 220px;
     background-color: white; z-index:9999; font-size:14px;
     border:2px solid grey; padding: 10px;
     ">
     <strong>Top 3 Most Expensive Counties</strong><br>
     <i class="fa fa-star" style="color: red;"></i> 1st Most Expensive<br>
     <i class="fa fa-star" style="color: darkred;"></i> 2nd Most Expensive<br>
     <i class="fa fa-star" style="color: orange;"></i> 3rd Most Expensive<br>
     <br>
     <strong>Price Heatmap</strong><br>
     <i style="background: rgba(255, 0, 0, 0.6); width: 18px; height: 18px; float: left; margin-right: 8px;"></i> High<br>
     <i style="background: rgba(255, 165, 0, 0.6); width: 18px; height: 18px; float: left; margin-right: 8px;"></i> Medium<br>
     <i style="background: rgba(0, 255, 0, 0.6); width: 18px; height: 18px; float: left; margin-right: 8px;"></i> Low<br>
     </div>
     '''
    folium_map.get_root().html.add_child(folium.Element(legend_html))

    return folium_map

In [ ]:

heat_map = create_price_heatmap(df_coordinates_county_price)
heat_map.save('heat_map.html')

In [ ]:

def outlier_thresholds(dataframe, col_name, q1=0.25, q3=0.75):
    quartile1 = dataframe[col_name].quantile(q1)
    quartile3 = dataframe[col_name].quantile(q3)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

In [ ]:

def check_outlier(dataframe, col_name):
    low_limit, up_limit = outlier_thresholds(dataframe, col_name)
    if dataframe[(dataframe[col_name] > up_limit) | (dataframe[col_name] < low_limit)].any(axis=None):
        return True
    else:
        return False

In [ ]:

def remove_outlier(dataframe, col_name):
    low_limit, up_limit = outlier_thresholds(dataframe, col_name)
    df_without_outliers = dataframe[~((dataframe[col_name] < low_limit) | (dataframe[col_name] > up_limit))]
    return df_without_outliers

In [ ]:

def replace_with_thresholds(dataframe, col):
    low_limit, up_limit = outlier_thresholds(dataframe, col)
    dataframe[col] = dataframe[col].apply(lambda x: low_limit if x < low_limit else (up_limit if x > up_limit else x))
    return dataframe

In [ ]:
def outlier_thresholds_per_categoric(dataframe, col_name, cat_name, q1=0.25, q3=0.75):
    outlier_data = []
    for cat in dataframe[cat_name].unique():
        filtered_df = dataframe[dataframe[cat_name] == cat]
        total_length = filtered_df.shape[0]
        low_limit, up_limit = outlier_thresholds(filtered_df, col_name, q1=q1, q3=q3)

        low_limit_length = filtered_df[filtered_df[col_name] < low_limit].shape[0]
        up_limit_length = filtered_df[filtered_df[col_name] > up_limit].shape[0]

        low_limit_ratio = low_limit_length / total_length
        up_limit_ratio = up_limit_length / total_length

        outlier_data.append((cat, low_limit, up_limit, low_limit_length, up_limit_length,
                             low_limit_ratio, up_limit_ratio))

    outlier_df = pd.DataFrame(outlier_data, columns=[cat_name, 'Low Limit', 'Up Limit',
                                                     'Low Limit Length', 'Up Limit Length',
                                                     'Low Limit Ratio', 'Up Limit Ratio'])

    outlier_df['Low Limit'] = outlier_df['Low Limit'].map('{:.3f}'.format)
    outlier_df['Up Limit'] = outlier_df['Up Limit'].map('{:.3f}'.format)
    outlier_df['Low Limit Ratio'] = outlier_df['Low Limit Ratio'].map('{:.3f}'.format)
    outlier_df['Up Limit Ratio'] = outlier_df['Up Limit Ratio'].map('{:.3f}'.format)

    return outlier_df

In [ ]:
df['price_change'] = pd.to_numeric(df['price_change'], errors='coerce')


In [ ]:

outlier_df_per_county_price_change = outlier_thresholds_per_categoric(df, 'price_change', 'county')

In [ ]:

outlier_df_per_county_price_change.sort_values(by = 'Up Limit Ratio', ascending = False)[:20]

In [ ]:

outlier_df_per_county_fee = outlier_thresholds_per_categoric(df, 'fee', 'county')

In [ ]:

outlier_df_per_county_fee.sort_values(by = 'Up Limit Ratio', ascending = False)[:20]

In [ ]:
translation_dict = {
    'Villa': 'House',
    'Lägenhet': 'Apartment',
    'Tomt': 'Plot',
    'Radhus': 'Townhouse',
    'Parhus': 'Terraced house',
    'Vinterbonat fritidshus': 'Winterized holiday home',
    'Övrig': 'Other',
    'Kedjehus': 'Linked house',
    'Par-/kedje-/radhus': 'Row/Terrace/Townhouse',
    'Fritidshus': 'Holiday home',
    'Gård utan jordbruk': 'Farm without agriculture',
    'Gård/skog' : 'Farm/Forest',
    'Gård med jordbruk' : 'Farm with agriculture',
    'Fritidsboende': 'Holiday accommodation',
    'Gård med skogsbruk': 'Farm with forestry'
}


df['property_type_en'] = df['property_type'].map(translation_dict)
df= df.drop(["property_type"], axis=1)
df.head()

In [ ]:
translation_dict = {
    'Äganderätt': 'Freehold',
    'Bostadsrätt': 'Cooperative apartment',
    'Annat': 'Other',
    'Andelsboende': 'Cooperative housing',
    'Tomträtt': 'Land lease',
    'Andel i bostadsförening': 'Share in housing association',
    'Information saknas': 'Information missing',
    'Hyresrätt': 'Rental'
}

# Çeviriyi uygula
df['ownership_type_en'] = df['ownership_type'].map(translation_dict)
df= df.drop(["ownership_type"], axis=1)
df.head()

In [ ]:
# Sütun adını değiştirme
df.rename(columns={'broker': 'estate_agent',
                   'property_type_en': 'property_type',
                   'ownership_type_en': 'ownership_type'}, inplace=True)
df.head()

In [ ]:
def translate_story(story):
    if isinstance(story, str):  # Sadece string değerleri işle
        # Kat ve kat sayısı için dönüşüm
        if ' av ' in story:
            parts = story.split(', ')
            for part in parts:
                if ' av ' in part:
                    floor_info = part.split(' av ')
                    if len(floor_info) == 2:
                        floor = floor_info[0].strip()
                        total_floors = floor_info[1].strip()
                        translated_part = f'{floor}th floor of {total_floors}'
                        story = story.replace(part, translated_part)
    # Asansör durumu için dönüşüm
    if 'hiss finns ej' in story:
        story = story.replace('hiss finns ej', 'no elevator')
    elif 'hiss finns' in story:
        story = story.replace('hiss finns', 'elevator available')

    return story

In [ ]:
# NaN değerlerini kontrol etmek ve apply fonksiyonunu güvenli kullanmak için fillna ile NaN değerlerini dolduruyoruz.
df['story'].fillna('', inplace=True)
# story sütununu çevir
df['story'] = df['story'].apply(translate_story)
df.head()

In [ ]:
df['story'].fillna('unknown', inplace=True)
df['story'].isnull().sum()

In [ ]:
df['story'].isnull().sum()
df['balcony'].fillna('no info', inplace=True)
df['balcony'].isnull().sum()


In [ ]:
df= df.drop(["floor"], axis=1)
df.head()

In [ ]:
# 'balcony' değişkenini çevir
df['balcony'] = df['balcony'].map({'Ja': 'Yes', 'Nej': 'No'})
df['balcony']

In [ ]:
df.to_csv('hemnet.csv', index = False)

In [ ]:
df_ = pd.read_csv("hemnet.csv")
df = df_.copy()
df.head()

In [ ]:
df['construction_year'] =df['construction_year'].astype(int)


In [ ]:
cat_cols, num_cols, cat_but_car, date_cols = grab_col_names(df, cat_th = 10, car_th = 300)
cat_cols

In [ ]:
num_cols

In [ ]:
df.head()


In [ ]:
one_hot_columns = ['property_type', 'housing_form', 'ownership_type', 'county','balcony']
drop_columns = ['street', 'build_year', 'land_area', 'area', 'latitude', 'longitude', 'operating_cost',
                 'association', 'estate_agent', 'story' ,'fee','url',
                    'sold_date','sold_year', 'sold_month','price_change','construction_year','living_area']

In [ ]:
dfd = pd.get_dummies(df, columns = one_hot_columns, dtype = int, drop_first = True)
dfd.head()

In [ ]:
dfd['price'] = dfd['price'] / 100000
dfd['wanted_price'] = dfd['wanted_price'] / 100000
dfd['fee'] = dfd['fee'] / 100000

dfd.drop(columns = drop_columns, inplace = True)

In [ ]:
dfd.head()


In [ ]:
# # price_change sütununun sayısal olup olmadığını kontrol etme ve NaN değerlerini temizleme
# dfd['price_change'] = pd.to_numeric(dfd['price_change'], errors='coerce')

In [ ]:
# dfd = dfd.dropna(subset=['price_change'])
# dfd.head()

In [ ]:
print(df[['price_change', 'property_type']].isnull().sum())


In [ ]:
df = df.dropna(subset=['price_change', 'property_type'])


In [ ]:
sns.boxplot(df, x = 'price_change', y = 'property_type')
plt.xlim(0, 0.5)
plt.show(block = True)

In [ ]:
X = dfd.drop(columns = ['price'])
y = dfd['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=40)

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)
r2_lr = r2_score(y_test, lr_pred)
mae_lr = mean_absolute_error(y_test, lr_pred)
mse_lr = mean_squared_error(y_test, lr_pred)
rmse_lr = mean_squared_error(y_test, lr_pred, squared = False)

In [ ]:
print(f'r2: {r2_lr:.4f}')
print(f'mae: {mae_lr:.4f}')
print(f'mse: {mse_lr:.4f}')
print(f'rmse: {rmse_lr:.4f}')

In [ ]:
ridge = Ridge()
ridge.fit(X_train, y_train)
ridge_pred = ridge.predict(X_test)
r2_ridge = r2_score(y_test, ridge_pred)
mae_ridge = mean_absolute_error(y_test, ridge_pred)
mse_ridge = mean_squared_error(y_test, ridge_pred)
rmse_ridge = mean_squared_error(y_test, ridge_pred, squared = False)

In [ ]:
print(f'r2: {r2_ridge:.4f}')
print(f'mae: {mae_ridge:.4f}')
print(f'mse: {mse_ridge:.4f}')
print(f'rmse: {rmse_ridge:.4f}')

In [ ]:

br = BayesianRidge()
br.fit(X_train, y_train)
br_pred = br.predict(X_test)
r2_br = r2_score(y_test, br_pred)
mea_br = mean_absolute_error(y_test, br_pred)
mse_br = mean_squared_error(y_test, br_pred)
rmse_br = mean_squared_error(y_test, br_pred, squared = False)

In [ ]:
print(f'r2: {r2_br:.4f}')
print(f'mae: {mea_br:.4f}')
print(f'mse: {mse_br:.4f}')
print(f'rmse: {rmse_br:.4f}')

In [ ]:

en = ElasticNet()
en.fit(X_train, y_train)
en_pred = en.predict(X_test)
r2_en = r2_score(y_test, en_pred)
mae_en = mean_absolute_error(y_test, en_pred)
mse_en = mean_squared_error(y_test, en_pred)
rmse_en = mean_squared_error(y_test, en_pred, squared=False)

In [ ]:
print(f'r2: {r2_en:.4f}')
print(f'mae: {mae_en:.4f}')
print(f'mse: {mse_en:.4f}')
print(f'rmse: {rmse_en:.4f}')

In [ ]:

xgb = XGBRegressor(max_depth = 12, learning_rate = 0.15, reg_lambda = 10)
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)
r2_xgb=r2_score(y_test,xgb_pred)
mae_xgb=mean_absolute_error(y_test,xgb_pred)
mse_xgb=mean_squared_error(y_test,xgb_pred)
rmse_xgb=mean_squared_error(y_test,xgb_pred,squared=False)

In [ ]:
print(f'r2: {r2_xgb:.4f}')
print(f'mae: {mae_xgb:.4f}')
print(f'mse: {mse_xgb:.4f}')
print(f'rmse: {rmse_xgb:.4f}')

In [ ]:

lgb = LGBMRegressor(n_estimators=200, random_state=40, learning_rate = 0.5, verbose = -1)
lgb.fit(X_train, y_train)
lgb_pred = lgb.predict(X_test)
r2_lgb=r2_score(y_test,lgb_pred)
mae_lgb=mean_absolute_error(y_test,lgb_pred)
mse_lgb=mean_squared_error(y_test,lgb_pred)
rmse_lgb=mean_squared_error(y_test,lgb_pred,squared=False)

In [ ]:
print(f'r2: {r2_lgb:.4f}')
print(f'mae: {mae_lgb:.4f}')
print(f'mse: {mse_lgb:.4f}')
print(f'rmse: {rmse_lgb:.4f}')

In [ ]:

cb = CatBoostRegressor(verbose = False, depth = 9, loss_function='RMSE', iterations = 1200)
cb.fit(X_train, y_train)
cb_pred = cb.predict(X_test)
r2_cb=r2_score(y_test,cb_pred)
mae_cb=mean_absolute_error(y_test,cb_pred)
mse_cb=mean_squared_error(y_test,cb_pred)
rmse_cb=mean_squared_error(y_test,cb_pred,squared=False)

In [ ]:
print(f'r2: {r2_cb:.4f}')
print(f'mae: {mae_cb:.4f}')
print(f'mse: {mse_cb:.4f}')
print(f'rmse: {rmse_cb:.4f}')

In [ ]:

lasso = Lasso()
lasso.fit(X_train, y_train)
lasso_pred = lasso.predict(X_test)
r2_lasso=r2_score(y_test,lasso_pred)
mae_lasso=mean_absolute_error(y_test,lasso_pred)
mse_lasso=mean_squared_error(y_test,lasso_pred)
rmse_lasso=mean_squared_error(y_test,lasso_pred,squared=False)

In [ ]:
print(f'r2: {r2_lasso:.4f}')
print(f'mae: {mae_lasso:.4f}')
print(f'mse: {mse_lasso:.4f}')
print(f'rmse: {rmse_lasso:.4f}')

In [ ]:

from sklearn.linear_model import GammaRegressor, PoissonRegressor, HuberRegressor
hr = HuberRegressor()
hr.fit(X_train, y_train) #runs very slowly
hr_predict = hr.predict(X_test)
r2_hr = r2_score(y_test, hr_predict)
mae_hr = mean_absolute_error(y_test, hr_predict)
mse_hr = mean_squared_error(y_test, hr_predict)
rmse_hr = mean_squared_error(y_test, hr_predict, squared=False)

In [ ]:
print(f'r2: {r2_hr:.4f}')
print(f'mae: {mae_hr:.4f}')
print(f'mse: {mse_hr:.4f}')
print(f'rmse: {rmse_hr:.4f}')

In [ ]:

from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
gbr = GradientBoostingRegressor()
gbr.fit(X_train, y_train) #runs very slowly
gbr_pred = gbr.predict(X_test)
r2_gbr = r2_score(y_test, gbr_pred)
mae_gbr = mean_absolute_error(y_test, gbr_pred)
mse_gbr = mean_squared_error(y_test, gbr_pred)
rmse_gbr = mean_squared_error(y_test, gbr_pred, squared=False)

In [ ]:
print(f'r2: {r2_gbr:.4f}')
print(f'mae: {mae_gbr:.4f}')
print(f'mse: {mse_gbr:.4f}')
print(f'rmse: {rmse_gbr:.4f}')

In [ ]:

ab = AdaBoostRegressor()
ab.fit(X_train, y_train) #runs very slowly
ab_pred = ab.predict(X_test)
r2_ab = r2_score(y_test, ab_pred)
mae_ab = mean_absolute_error(y_test, ab_pred)
mse_ab = mean_squared_error(y_test, ab_pred)
rmse_ab = mean_squared_error(y_test, ab_pred, squared=False)

In [ ]:
print(f'r2: {r2_ab:.4f}')
print(f'mae: {mae_ab:.4f}')
print(f'mse: {mse_ab:.4f}')
print(f'rmse: {rmse_ab:.4f}')


In [ ]:
df_ = pd.read_csv("hemnet.csv")
df = df_.copy()


In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

In [ ]:
# Kullanıcıdan veri alma (örnek)
min_price = input("Aradığınız minimum fiyat: ")
max_price = input("Aradığınız maksimum fiyat: ")
user_county = input("Aradığınız şehir: ")
property_type = input("Aradığınız mülk tipi: ")
min_living_area = input("Aradığınız minimum yaşam alanı (m²): ")
max_living_area = input("Aradığınız maksimum yaşam alanı (m²): ")
min_rooms = input("Aradığınız minimum oda sayısı: ")
max_rooms = input("Aradığınız maksimum oda sayısı: ")
min_build_year = input("Aradığınız minimum yapım yılı: ")
max_build_year = input("Aradığınız maksimum yapım yılı: ")
user_balcony = input("Balkon tercihiniz (1 = Evet, 0 = Hayır, boş bırakın): ")


In [ ]:
# Kullanıcı profili
user_profile = {
    'min_price': float(min_price) if min_price else None,
    'max_price': float(max_price) if max_price else None,
    'county': user_county if user_county else None,
    'property_type': property_type if property_type else None,
    'min_living_area': float(min_living_area) if min_living_area else None,
    'max_living_area': float(max_living_area) if max_living_area else None,
    'min_rooms': int(min_rooms) if min_rooms else None,
    'max_rooms': int(max_rooms) if max_rooms else None,
    'min_build_year': int(min_build_year) if min_build_year else None,
    'max_build_year': int(max_build_year) if max_build_year else None,
    'balcony': int(user_balcony) if user_balcony else None
}
user_profile

In [ ]:
# Şehirleri sayısal değerlere dönüştürmek için bir sözlük oluşturun
unique_counties = df['county'].unique()
county_mapping = {county: idx for idx, county in enumerate(unique_counties)}


In [ ]:
# Emlak verisini ve kullanıcı profilini dönüştürme
df['county'] = df['county'].map(county_mapping)
user_profile['county'] = county_mapping.get(user_profile['county'], -1) if user_profile['county'] else None


In [ ]:
# Veriyi ve kullanıcı profilini filtreleme
filters = [
    (df['price'] >= user_profile['min_price']) if user_profile['min_price'] else True,
    (df['price'] <= user_profile['max_price']) if user_profile['max_price'] else True,
    (df['rooms'] >= user_profile['min_rooms']) if user_profile['min_rooms'] else True,
    (df['rooms'] <= user_profile['max_rooms']) if user_profile['max_rooms'] else True,
    (df['build_year'] >= user_profile['min_build_year']) if user_profile['min_build_year'] else True,
    (df['build_year'] <= user_profile['max_build_year']) if user_profile['max_build_year'] else True,
    (df['balcony'] == user_profile['balcony']) if user_profile['balcony'] is not None else True,
    (df['county'] == user_profile['county']) if user_profile['county'] is not None else True,
    (df['property_type'] == user_profile['property_type']) if user_profile['property_type'] else True,
    (df['living_area'] >= user_profile['min_living_area']) if user_profile['min_living_area'] else True,
    (df['living_area'] <= user_profile['max_living_area']) if user_profile['max_living_area'] else True
]
filters


In [ ]:
filtered_df = df.loc[filters[0]]
for filter_condition in filters[1:]:
    if not isinstance(filter_condition, bool):
        filtered_df = filtered_df.loc[filter_condition]
filtered_df.head()

In [ ]:
# Filtrelenmiş veri olup olmadığını kontrol etme
if filtered_df.empty:
    print("Belirtilen kriterlere uygun ev bulunamadı.")
else:
    # Filtrelenmiş veriyi ve kullanıcı profilini standartlaştırma
    features = filtered_df[['price', 'rooms', 'build_year', 'county', 'balcony', 'living_area']]
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features)

    # Kullanıcı profilini standartlaştırma
    user_profile_values = [
        user_profile['max_price'] if user_profile['max_price'] is not None else df['price'].max(),
        user_profile['max_rooms'] if user_profile['max_rooms'] is not None else df['rooms'].max(),
        user_profile['max_build_year'] if user_profile['max_build_year'] is not None else df['build_year'].max(),
        user_profile['county'] if user_profile['county'] is not None else -1,
        user_profile['balcony'] if user_profile['balcony'] is not None else df['balcony'].mode()[0],
        user_profile['max_living_area'] if user_profile['max_living_area'] is not None else df['living_area'].max()
    ]

    user_profile_scaled = scaler.transform([user_profile_values])

    # Benzerlik hesaplama
    similarities = cosine_similarity(user_profile_scaled, scaled_features)
    filtered_df['similarity'] = similarities[0]

    # Öneri listesi
    recommendations = filtered_df.sort_values(by='similarity', ascending=False)
    print("Önerilen Evler:")
    print(recommendations[['street', 'price', 'rooms', 'build_year', 'balcony', 'living_area']])

In [ ]:
# Kullanıcıdan değerleri almak için bir fonksiyon
def get_user_input():
    property_type = input("Enter property type (Apartment/House): ").capitalize()
    price_range = input("Enter price range (e.g., 2700000-3100000): ")
    min_price, max_price = map(int, price_range.split('-'))
    county = input("Enter county (e.g., Stockholms): ").capitalize()
    
    user_input = {
        'property_type': property_type,
        'min_price': min_price,
        'max_price': max_price,
        'county': county
    }
    
    return user_input

# Kullanıcıdan değerleri al
user_input = get_user_input()

# Kullanıcı girdilerine göre veri filtresi uygulama
filtered_df = df[
    (df['property_type'] == user_input['property_type']) &
    (df['price'] >= user_input['min_price']) &
    (df['price'] <= user_input['max_price']) &
    (df['county'] == user_input['county'])
]

# Önerileri gösterme
if not filtered_df.empty:
    print(f"Found {len(filtered_df)} matching properties:")
    for idx, row in filtered_df.iterrows():
        print(f"\nStreet: {row['street']}")
        print(f"Price: {row['price']}")
        print(f"County: {row['county']}")
        print(f"Living Area: {row['living_area']}")
        print(f"Property Type: {row['property_type']}")
        print(f"URL: {row['url']}")
else:
    print("No matching properties found.")

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

# Örnek emlak verisi
data = {
    'street': ['Östra Hamngatan 2', 'Läckövägen 26 *ACCEPTERAT PRIS*'],
    'price': [2750000, 3650000],
    'county': ['Västerås', 'Stockholms'],
    'rooms': [2, 3],
    'build_year': [2011, 1943],
    'balcony': [1, 0]  # Evet = 1, Hayır = 0
}

df = pd.DataFrame(data)

# Kullanıcıdan veri alma (örnek)
min_price = float(input("Aradığınız minimum fiyat: "))
max_price = float(input("Aradığınız maksimum fiyat: "))
user_county = input("Aradığınız şehir: ")
min_rooms = int(input("Aradığınız minimum oda sayısı: "))
max_rooms = int(input("Aradığınız maksimum oda sayısı: "))
min_build_year = int(input("Aradığınız minimum yapım yılı: "))
max_build_year = int(input("Aradığınız maksimum yapım yılı: "))
user_balcony = int(input("Balkon tercihiniz (1 = Evet, 0 = Hayır): "))

# Kullanıcı profili
user_profile = {
    'min_price': min_price,
    'max_price': max_price,
    'county': user_county,
    'min_rooms': min_rooms,
    'max_rooms': max_rooms,
    'min_build_year': min_build_year,
    'max_build_year': max_build_year,
    'balcony': user_balcony
}

# Şehirleri sayısal değerlere dönüştürmek için bir sözlük oluşturun
unique_counties = df['county'].unique()
county_mapping = {county: idx for idx, county in enumerate(unique_counties)}

# Emlak verisini ve kullanıcı profilini dönüştürme
df['county'] = df['county'].map(county_mapping)
user_profile['county'] = county_mapping.get(user_profile['county'], -1)  # -1 veya başka bir değer geçerli değilse

# Veriyi ve kullanıcı profilini filtreleme
filtered_df = df[
    (df['price'] >= user_profile['min_price']) &
    (df['price'] <= user_profile['max_price']) &
    (df['rooms'] >= user_profile['min_rooms']) &
    (df['rooms'] <= user_profile['max_rooms']) &
    (df['build_year'] >= user_profile['min_build_year']) &
    (df['build_year'] <= user_profile['max_build_year']) &
    (df['balcony'] == user_profile['balcony'])
]

# Filtrelenmiş veri olup olmadığını kontrol etme
if filtered_df.empty:
    print("Belirtilen kriterlere uygun ev bulunamadı.")
else:
    # Filtrelenmiş veriyi ve kullanıcı profilini standartlaştırma
    features = filtered_df[['price', 'rooms', 'build_year', 'county', 'balcony']]
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features)

    # Kullanıcı profilini standartlaştırma
    user_profile_scaled = scaler.transform([[user_profile['max_price'], user_profile['max_rooms'], user_profile['max_build_year'], user_profile['county'], user_profile['balcony']]])

    # Benzerlik hesaplama
    similarities = cosine_similarity(user_profile_scaled, scaled_features)
    filtered_df['similarity'] = similarities[0]

    # Öneri listesi
    recommendations = filtered_df.sort_values(by='similarity', ascending=False)
    print("Önerilen Evler:")
    print(recommendations[['street', 'price', 'rooms', 'build_year', 'balcony']])


In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

df = pd.read_csv("hemnet.csv")
# build_year sütununu yıl olarak parçalama
df['build_year'] = pd.to_datetime(df['build_year']).dt.year
df.head()

In [ ]:

# Kullanıcıdan veri alma (örnek)
min_price = input("Aradığınız minimum fiyat: ")
max_price = input("Aradığınız maksimum fiyat: ")
user_county = input("Aradığınız şehir: ")
property_type = input("Aradığınız mülk tipi: ")
min_living_area = input("Aradığınız minimum yaşam alanı (m²): ")
max_living_area = input("Aradığınız maksimum yaşam alanı (m²): ")
min_rooms = input("Aradığınız minimum oda sayısı: ")
max_rooms = input("Aradığınız maksimum oda sayısı: ")
min_build_year = input("Aradığınız minimum yapım yılı: ")
max_build_year = input("Aradığınız maksimum yapım yılı: ")
user_balcony = input("Balkon tercihiniz (1 = Evet, 0 = Hayır, boş bırakın): ")

# Kullanıcı profili
user_profile = {
    'min_price': float(min_price) if min_price else None,
    'max_price': float(max_price) if max_price else None,
    'county': user_county if user_county else None,
    'property_type': property_type if property_type else None,
    'min_living_area': float(min_living_area) if min_living_area else None,
    'max_living_area': float(max_living_area) if max_living_area else None,
    'min_rooms': int(min_rooms) if min_rooms else None,
    'max_rooms': int(max_rooms) if max_rooms else None,
    'min_build_year': int(min_build_year) if min_build_year else None,
    'max_build_year': int(max_build_year) if max_build_year else None,
    'balcony': int(user_balcony) if user_balcony else None
}

# Şehirleri sayısal değerlere dönüştürmek için bir sözlük oluşturun
unique_counties = df['county'].unique()
county_mapping = {county: idx for idx, county in enumerate(unique_counties)}

# Emlak verisini ve kullanıcı profilini dönüştürme
df['county'] = df['county'].map(county_mapping)
user_profile['county'] = county_mapping.get(user_profile['county'], -1) if user_profile['county'] else None

# Veriyi ve kullanıcı profilini filtreleme
filters = [
    (df['price'] >= user_profile['min_price']) if user_profile['min_price'] is not None else pd.Series([True] * len(df)),
    (df['price'] <= user_profile['max_price']) if user_profile['max_price'] is not None else pd.Series([True] * len(df)),
    (df['rooms'] >= user_profile['min_rooms']) if user_profile['min_rooms'] is not None else pd.Series([True] * len(df)),
    (df['rooms'] <= user_profile['max_rooms']) if user_profile['max_rooms'] is not None else pd.Series([True] * len(df)),
    (df['build_year'] >= user_profile['min_build_year']) if user_profile['min_build_year'] is not None else pd.Series([True] * len(df)),
    (df['build_year'] <= user_profile['max_build_year']) if user_profile['max_build_year'] is not None else pd.Series([True] * len(df)),
    (df['balcony'] == user_profile['balcony']) if user_profile['balcony'] is not None else pd.Series([True] * len(df)),
    (df['county'] == user_profile['county']) if user_profile['county'] is not None else pd.Series([True] * len(df)),
    (df['property_type'] == user_profile['property_type']) if user_profile['property_type'] is not None else pd.Series([True] * len(df)),
    (df['living_area'] >= user_profile['min_living_area']) if user_profile['min_living_area'] is not None else pd.Series([True] * len(df)),
    (df['living_area'] <= user_profile['max_living_area']) if user_profile['max_living_area'] is not None else pd.Series([True] * len(df))
]

# Tüm filtreleri birleştirme
combined_filters = filters[0]
for filter_condition in filters[1:]:
    combined_filters &= filter_condition

filtered_df = df[combined_filters]

# Ara sonuçları kontrol etmek için veri çerçevesinin başını yazdır
print(filtered_df.head())

In [ ]:

# Filtrelenmiş veri olup olmadığını kontrol etme
if filtered_df.empty:
    print("Belirtilen kriterlere uygun ev bulunamadı.")
else:
    # Filtrelenmiş veriyi ve kullanıcı profilini standartlaştırma
    features = filtered_df[['price', 'rooms', 'build_year', 'county', 'balcony', 'living_area']]
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features)

    # Kullanıcı profilini standartlaştırma
    user_profile_values = [
        user_profile['max_price'] if user_profile['max_price'] is not None else df['price'].max(),
        user_profile['max_rooms'] if user_profile['max_rooms'] is not None else df['rooms'].max(),
        user_profile['max_build_year'] if user_profile['max_build_year'] is not None else df['build_year'].max(),
        user_profile['county'] if user_profile['county'] is not None else -1,
        user_profile['balcony'] if user_profile['balcony'] is not None else df['balcony'].mode()[0],
        user_profile['max_living_area'] if user_profile['max_living_area'] is not None else df['living_area'].max()
    ]

    user_profile_scaled = scaler.transform([user_profile_values])

    # Benzerlik hesaplama
    similarities = cosine_similarity(user_profile_scaled, scaled_features)
    filtered_df['similarity'] = similarities[0]

    # Öneri listesi
    recommendations = filtered_df.sort_values(by='similarity', ascending=False)
    recommendations


In [ ]:
import pandas as pd

df = pd.read_csv("hemnet.csv")
# build_year sütununu yıl olarak parçalama
df['build_year'] = pd.to_datetime(df['build_year']).dt.year
df.head()

In [ ]:
def get_user_input():
    try:
        price_min = float(input("Minimum fiyat (boş bırakabilirsiniz): ") or -float('inf'))
    except ValueError:
        price_min = -float('inf')
    
    try:
        price_max = float(input("Maksimum fiyat (boş bırakabilirsiniz): ") or float('inf'))
    except ValueError:
        price_max = float('inf')

    property_type = input("Mülk tipi (Apartment/House) (boş bırakabilirsiniz): ").strip() or None
    try:
        living_area_min = float(input("Minimum m2 (boş bırakabilirsiniz): ") or 0)
    except ValueError:
        living_area_min = 0

    try:
        living_area_max = float(input("Maksimum m2 (boş bırakabilirsiniz): ") or float('inf'))
    except ValueError:
        living_area_max = float('inf')

    balcony = input("Balkon istiyor musunuz? (Evet/Hayır) (boş bırakabilirsiniz): ").strip() or None
    city = input("Şehir (boş bırakabilirsiniz): ").strip() or None
    
    return price_min, price_max, property_type, living_area_min, living_area_max, balcony, city


In [ ]:
def filter_properties(df, price_min, price_max, property_type, living_area_min, living_area_max, balcony, city):
    filtered_df = df[
        (df['price'] >= price_min) &
        (df['price'] <= price_max) &
        (df['living_area'] >= living_area_min) &
        (df['living_area'] <= living_area_max)
    ]
    
    if property_type:
        filtered_df = filtered_df[filtered_df['property_type'].str.contains(property_type, case=False)]
    
    if balcony == 'Evet':
        filtered_df = filtered_df[filtered_df['balcony'] == 'Ja']
    elif balcony == 'Hayır':
        filtered_df = filtered_df[filtered_df['balcony'] != 'Ja']
    
    if city:
        filtered_df = filtered_df[filtered_df['county'].str.contains(city, case=False)]
    
    return filtered_df

# Kullanıcı girdilerini alma
price_min, price_max, property_type, living_area_min, living_area_max, balcony, city = get_user_input()

# Verileri filtreleme
filtered_df = filter_properties(df, price_min, price_max, property_type, living_area_min, living_area_max, balcony, city)
print(filtered_df)


In [ ]:

# Sonuçları CSV dosyasına kaydetme
filtered_df.to_csv('filtered_properties.csv', index=False)

print("Önerilen evler 'filtered_properties.csv' dosyasına kaydedildi.")

In [ ]:
import streamlit as st

def filter_properties(df, price_min, price_max, property_type, living_area_min, living_area_max, balcony, city):
    filtered_df = df[
        (df['price'] >= price_min) &
        (df['price'] <= price_max) &
        (df['living_area'] >= living_area_min) &
        (df['living_area'] <= living_area_max)
    ]
    
    if property_type:
        filtered_df = filtered_df[filtered_df['property_type'].str.contains(property_type, case=False)]
    
    if balcony == 'Evet':
        filtered_df = filtered_df[filtered_df['balcony'] == 'Ja']
    elif balcony == 'Hayır':
        filtered_df = filtered_df[filtered_df['balcony'] != 'Ja']
    
    if city:
        filtered_df = filtered_df[filtered_df['county'].str.contains(city, case=False)]
    
    return filtered_df

# Streamlit uygulaması
st.title("Ev Öneri Sistemi")

price_min = st.number_input("Minimum fiyat", min_value=0.0, value=0.0, step=10000.0)
price_max = st.number_input("Maksimum fiyat", min_value=0.0, value=10000000.0, step=10000.0)
property_type = st.selectbox("Mülk tipi", ["Apartment", "House", "Any"], index=2)
living_area_min = st.number_input("Minimum m2", min_value=0.0, value=0.0, step=1.0)
living_area_max = st.number_input("Maksimum m2", min_value=0.0, value=1000.0, step=1.0)
balcony = st.selectbox("Balkon istiyor musunuz?", ["Evet", "Hayır", "Farketmez"], index=2)
city = st.text_input("Şehir")

if st.button("Evleri Listele"):
    property_type = None if property_type == "Any" else property_type
    balcony = None if balcony == "Farketmez" else balcony
    filtered_df = filter_properties(df, price_min, price_max, property_type, living_area_min, living_area_max, balcony, city)
    
    st.write("Önerilen Evler:")
    st.write(filtered_df)
    
    csv = filtered_df.to_csv(index=False).encode('utf-8')
    st.download_button(
        label="Önerilen evleri CSV olarak indir",
        data=csv,
        file_name='filtered_properties.csv',
        mime='text/csv',
    )